In [22]:
import pandas as pd
import glob


data_frames = []

for file_path in file_paths:
    df = pd.read_csv(file_path)
    data_frames.append(df)

merged_data = pd.concat(data_frames, ignore_index=True)
merged_data.to_csv('merged_data.csv', index=False)

merged_data.head()

,title,company,job_post_id,company_info,address,employment_type,seniority,min_experience,job_category,has_company_logo,salary,salary_type,num_of_applications,last_posted_date,expiry_date,description
0,IT Key Account Manager - EU,RK RECRUITMENT PTE. LTD.,MCF-2023-0729336,RK Recruitment is a workforce solutions provid...,NaN,Full Time,Manager,2 years exp,"Information Technology, Sales / Retail",1,"['$3,500to$4,500']",Monthly,0 application,Posted 24 Sep 2023,Closing on 24 Oct 2023,Roles & Responsibilities\r\nBenefits Summary:\...
1,Account Manager - Life Science / Biotech,TALENT TRADER GROUP PTE. LTD.,MCF-2023-0728383,TALENT TRADER GROUP PTE. LTD. Talent Trader is...,NaN,"Permanent, Full Time",Executive,2 years exp,"Healthcare / Pharmaceutical, Sales / Retail",1,"['$3,000to$6,000']",Monthly,0 application,Posted 23 Sep 2023,Closing on 23 Oct 2023,"Roles & Responsibilities\r\nProspect, develop ..."
2,Senior Customer Success Manager,VODAFONE ENTERPRISE SINGAPORE PTE. LTD.,MCF-2023-0658066,"Vodafone Group is a Global organisation, commi...","ASIA SQUARE TOWER 2, 12 MARINA VIEW 018961",Contract,Manager,6 years exp,Telecommunications,1,"['$9,700to$13,000']",Monthly,138 applications,Posted 28 Aug 2023,Closing on 27 Sep 2023,Roles & Responsibilities\r\nKey accountabiliti...
3,Key Account Manager |Freight Forwarding - Hunt...,TALENT TRADER GROUP PTE. LTD.,MCF-2023-0728388,TALENT TRADER GROUP PTE. LTD. Talent Trader is...,NaN,"Permanent, Full Time",Manager,5 years exp,"Logistics / Supply Chain, Sales / Retail",1,"['$6,000to$9,000']",Monthly,0 application,Posted 23 Sep 2023,Closing on 23 Oct 2023,Roles & Responsibilities\r\nResponsibilities\r...
4,Account Manager [Sales / Audio Visual / B2B / ...,RK RECRUITMENT PTE. LTD.,MCF-2023-0728553,RK Recruitment is a workforce solutions provid...,NaN,Full Time,Senior Executive,1 year exp,"Admin / Secretarial, Sales / Retail",1,"['$3,500to$5,500']",Monthly,0 application,Posted 23 Sep 2023,Closing on 23 Oct 2023,Roles & Responsibilities\r\nJob Info:\r\nMon t...


In [31]:
file_path = 'merged_data.csv'
df = pd.read_csv(file_path)

# Clean the 'salary' column by removing dollar signs
df['salary'] = df['salary'].str.replace(r'[,\[\]\']', '', regex=True)
df['salary'] = df['salary'].str.replace('to', '-', regex=False)
df['salary'] = df['salary'].str.replace('$', '', regex=False)

df.rename(columns={'job_category': 'department'}, inplace=True)
df.rename(columns={'salary': 'salary_range'}, inplace=True)
df.rename(columns={'company_info': 'company_profile'}, inplace=True)
df.rename(columns={'seniority': 'required_experience'}, inplace=True)

df = df.assign(industry=df['department'])

# Keywords to check for in the 'description' column
keywords = ['WFH', 'work from home', 'telecommuting', 'telecommute', 'hybrid', 'remote']

df['telecommuting'] = (df['title'].str.contains('|'.join(keywords), case=False, na=False) | df['description'].str.contains('|'.join(keywords), case=False, na=False)).astype(int)

df['has_questions'] = df['description'].str.contains(r'\?', case=False, na=False).astype(int)

# Initialize empty lists for description, requirements, and benefits
descriptions = []
requirements = []
benefits = []

# Extract information from the 'description' column
for description_text in df['description']:
    sections = [section.strip() for section in description_text.split('\n') if section.strip()]

    description = ""
    requirement = ""
    benefit = ""
    
    section_titles = ["Roles & Responsibilities", "Key Responsibilities", "Qualifications", "MINIMUM QUALIFICATIONS", "QUALIFICATIONS", "Your Profile", "Profile", "What Do We Want", "Requirements", "Benefits"]

    current_section = None
    for section in sections:
        if any(title in section for title in section_titles):
            current_section = section
        else:
            if current_section == "Roles & Responsibilities" or current_section == "Key Responsibilities":
                description += section + '\n'
            elif current_section == "Qualifications" or current_section == "Requirements" \
            or current_section == "MINIMUM QUALIFICATIONS" or current_section == "QUALIFICATIONS" \
            or current_section == "Your Profile" or current_section == "Profile" or current_section == "What Do We Want":
                requirement += section + '\n'
            elif current_section == "Benefits":
                benefit += section + '\n'

    descriptions.append(description.strip())
    requirements.append(requirement.strip())
    benefits.append(benefit.strip())

df['description'] = descriptions
df['requirement'] = requirements
df['benefits'] = benefits

df = df[['job_post_id', 'title', 'address', 'department', 'salary_range', 'company_profile', 'description', 'requirement', 'benefits', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'min_experience', 'company', 'industry', 'salary_type']]

# Check for duplicate rows based on all columns except for 'job_post_id' (assuming 'job_post_id' is the unique identifier)
df.drop_duplicates(subset=df.columns.difference(['job_post_id']), inplace=True)

df.to_csv('cleaned_data.csv', index=False)

print(df.head(2))

        job_post_id                                     title address  \
0  MCF-2023-0729336               IT Key Account Manager - EU     NaN   
1  MCF-2023-0728383  Account Manager - Life Science / Biotech     NaN   

                                    department salary_range  \
0       Information Technology, Sales / Retail    3500-4500   
1  Healthcare / Pharmaceutical, Sales / Retail    3000-6000   

                                     company_profile  \
0  RK Recruitment is a workforce solutions provid...   
1  TALENT TRADER GROUP PTE. LTD. Talent Trader is...   

                                         description requirement benefits  \
0                                                                           
1  Prospect, develop and drive sales and marketin...                        

   telecommuting  has_company_logo  has_questions       employment_type  \
0              0                 1              0             Full Time   
1              0                 1     